In [1]:
import sys
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import json
sys.path.insert(0, '/root/siqi/Feature-Mining-for-Object-Detection')
from feature_mining.decision_tree import *


In [2]:
train_dataset = torch.load('/root/data/activation_outputs/train_data_29.pt')
# The activation outputs
train_data = train_dataset['data'].numpy()
print(train_data.shape)
train_data = train_data.reshape(train_data.shape[0], -1)
# The attributes associate with the activation outputs
train_attributes = train_dataset['label']

val_dataset = torch.load('/root/data/activation_outputs/val_data_29.pt')
# The activation outputs
val_data = val_dataset['data'].numpy()
val_data = val_data.reshape(val_data.shape[0], -1)
# The attributes associate with the activation outputs
val_attributes = val_dataset['label']

(2004, 90, 30, 52)


In [3]:
print(train_data.shape)
print(val_data.shape)

(2004, 140400)
(2004, 140400)


In [4]:
print(f'Activation output 0: {train_data[0]}')
print(f'Attributes 0: {train_attributes[0]}')

Activation output 0: [-0.18898432  0.32860994  0.04227519 ... -1.4035988  -0.8112181
 -0.8552464 ]
Attributes 0: {'cycle.with_rider', 'pedestrian.moving', 'pedestrian.standing', 'vehicle.moving'}


In [27]:
# Assign label
# target_attribute = 'pedestrian.moving'
# target_attribute = 'vehicle.parked'
# target_attribute = 'pedestrian'
target_attribute = 'vehicle'
train_label = [False] * len(train_data)

for i, attributes in enumerate(train_attributes):
    for attribute in attributes:
        if target_attribute in attribute:
            train_label[i] = True
            break

print(train_label.count(True), train_label.count(False))

val_label = [False] * len(val_data)

for i, attributes in enumerate(val_attributes):
    for attribute in attributes:
        if target_attribute in attribute:
            val_label[i] = True

print(val_label.count(True), val_label.count(False))

1616 388
1548 456


In [28]:
invariants = get_invariant(train_data, train_label)

Obtained all paths


100%|██████████| 81/81 [00:00<00:00, 209715.20it/s]


In [29]:
# print(invariants[0])
print(type(invariants))
print(len(invariants))
print(invariants.keys())
print(f'{len(invariants[True])} for Class True, {len(invariants[False])} for Class False')

<class 'dict'>
2
dict_keys([True, False])
34 for Class True, 47 for Class False


In [30]:
print('Class', True, invariants[True][0])
print()
print('Class', False, invariants[False][0])

Class True [[8607, 99766, 1615, 126036, 56537, 95695, 26677, 15639, 75465, 49649, 122423, 120283, 78035, 24079, 100644, 56211, 7971, 77811, 130096, 21999, 65187, 34323, 51873, 110143, 29905, 125405], ['<=', -6.795586109161377, '>', -1.0424776673316956, '>', -2.786024570465088, '<=', -4.470809698104858, '>', -7.610140562057495, '>', -0.6740290224552155, '<=', -0.2088850736618042, '<=', -5.060968637466431, '<=', -1.8092931509017944, '<=', 0.5216390192508698, '<=', -0.7047935724258423, '<=', -1.7720696926116943, '>', -6.5032243728637695, '<=', -2.5001354217529297, '>', -16.02127170562744, '>', -7.11587929725647, '<=', -1.8601446151733398, '>', -6.447704076766968, '<=', 0.770761251449585, '>', -6.694637775421143, '<=', -5.555307865142822, '>', -4.2399115562438965, '<=', 1.7600448727607727, '<=', -3.7868603467941284, '<=', -1.1621071696281433, '>', -6.090216636657715], 1214]

Class False [[8607, 99767, 24052, 63069, 73918, 15121, 76989, 63911, 117699, 86262, 27335], ['>', -6.795586109161377

In [31]:
print(len(invariants))
print(f'# of {1110} positive samples support true {invariants[True][0][-1]}')
print(f'# of {2004 - 1110} negative samples support false {invariants[False][0][-1]}')


2
# of 1110 positive samples support true 1214
# of 894 negative samples support false 161


In [32]:
validate(invariants[True][0], True, train_data, train_label)
print()
validate(invariants[False][0], False, train_data, train_label)

CLASS; True [[8607, 99766, 1615, 126036, 56537, 95695, 26677, 15639, 75465, 49649, 122423, 120283, 78035, 24079, 100644, 56211, 7971, 77811, 130096, 21999, 65187, 34323, 51873, 110143, 29905, 125405], ['<=', -6.795586109161377, '>', -1.0424776673316956, '>', -2.786024570465088, '<=', -4.470809698104858, '>', -7.610140562057495, '>', -0.6740290224552155, '<=', -0.2088850736618042, '<=', -5.060968637466431, '<=', -1.8092931509017944, '<=', 0.5216390192508698, '<=', -0.7047935724258423, '<=', -1.7720696926116943, '>', -6.5032243728637695, '<=', -2.5001354217529297, '>', -16.02127170562744, '>', -7.11587929725647, '<=', -1.8601446151733398, '>', -6.447704076766968, '<=', 0.770761251449585, '>', -6.694637775421143, '<=', -5.555307865142822, '>', -4.2399115562438965, '<=', 1.7600448727607727, '<=', -3.7868603467941284, '<=', -1.1621071696281433, '>', -6.090216636657715], 1214] ;Precision: 100.0 ;Recall: 75.12376237623762

CLASS; False [[8607, 99767, 24052, 63069, 73918, 15121, 76989, 63911, 

In [33]:
validate(invariants[True][0], True, val_data, val_label)
print()
validate(invariants[False][0], False, val_data, val_label)

CLASS; True [[8607, 99766, 1615, 126036, 56537, 95695, 26677, 15639, 75465, 49649, 122423, 120283, 78035, 24079, 100644, 56211, 7971, 77811, 130096, 21999, 65187, 34323, 51873, 110143, 29905, 125405], ['<=', -6.795586109161377, '>', -1.0424776673316956, '>', -2.786024570465088, '<=', -4.470809698104858, '>', -7.610140562057495, '>', -0.6740290224552155, '<=', -0.2088850736618042, '<=', -5.060968637466431, '<=', -1.8092931509017944, '<=', 0.5216390192508698, '<=', -0.7047935724258423, '<=', -1.7720696926116943, '>', -6.5032243728637695, '<=', -2.5001354217529297, '>', -16.02127170562744, '>', -7.11587929725647, '<=', -1.8601446151733398, '>', -6.447704076766968, '<=', 0.770761251449585, '>', -6.694637775421143, '<=', -5.555307865142822, '>', -4.2399115562438965, '<=', 1.7600448727607727, '<=', -3.7868603467941284, '<=', -1.1621071696281433, '>', -6.090216636657715], 1214] ;Precision: 87.95620437956204 ;Recall: 62.273901808785524

CLASS; False [[8607, 99767, 24052, 63069, 73918, 15121, 7